In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "LeNet5_ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[4] + "_EpochDown_LRdown_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[4] +  "_EpochDown_LRdown_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[4] + "_EpochDown_LRdown"

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC10(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)
    #篩選出ESC10作為訓練資料
    df_10 = df[df['esc10']==True]
    #取出類別，去除重複並排序
    classes = df_10['category'].unique()
    class_dict = {i:x for x,i in enumerate(classes)}
    #重新標label
    df_10['target'] = df_10['category'].map(class_dict)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df_10[df_10.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df_10))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold, setLR, setEpoch):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = setEpoch
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = setLR
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC10(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC10(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC10(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料
讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_7772\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)
C:\Users\test\AppData\Local\Temp\ipykernel_7772\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


C:\Users\test\AppData\Local\Temp\ipykernel_7772\2994481965.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)
C:\Users\test\AppData\Local\Temp\ipykernel_7772\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
讀取3秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_7772\2994481965.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)
C:\Users\test\AppData\Local\Temp\ipykernel_7772\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料
Progress: 100% - Completed
讀取5秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_7772\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)
C:\Users\test\AppData\Local\Temp\ipykernel_7772\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
Data Loading Time: 26.36 seconds
Progress: 100% - Completed
Data Loading Time: 26.36 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 10)

    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #學習率
    lr=0
    #epoch
    epoch=0
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #1s
            dataX = dataX5
            dataY = dataY5
            epoch = 60
            lr=0.0032
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
            epoch = 40
            lr=0.0016
        else:
            #5s
            dataX = dataX1
            dataY = dataY1
            epoch = 20
            lr=0.0008

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(trainFold=fold, setLR=lr, setEpoch=epoch)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 3.501004934310913 Train_acc: 11.250000%
Train Epoch: 2/60 Train_Loss: 2.25453519821167 Train_acc: 17.812500%
Train Epoch: 1/60 Train_Loss: 3.501004934310913 Train_acc: 11.250000%
Train Epoch: 2/60 Train_Loss: 2.25453519821167 Train_acc: 17.812500%
Train Epoch: 3/60 Train_Loss: 2.060777187347412 Train_acc: 22.500000%
Train Epoch: 4/60 Train_Loss: 1.9118129014968872 Train_acc: 25.000000%
Train Epoch: 5/60 Train_Loss: 1.7030256986618042 Train_acc: 29.062500

Train Epoch: 55/60 Train_Loss: 0.8728517889976501 Train_acc: 73.750000%
Train Epoch: 56/60 Train_Loss: 0.6616913676261902 Train_acc: 75.000000%
Train Epoch: 55/60 Train_Loss: 0.8728517889976501 Train_acc: 73.750000%
Train Epoch: 56/60 Train_Loss: 0.6616913676261902 Train_acc: 75.000000%
Train Epoch: 57/60 Train_Loss: 0.8054614663124084 Train_acc: 74.062500%
Train Epoch: 58/60 Train_Loss: 0.7512451410293579 Train_acc: 73.437500%
Train Epoch: 57/60 Train_Loss: 0.8054614663124084 Train_acc: 74.062500%
Train Epoch: 58/60 Train_Loss: 0.7512451410293579 Train_acc: 73.437500%
Train Epoch: 59/60 Train_Loss: 0.6709561347961426 Train_acc: 72.500000%
Train Epoch: 60/60 Train_Loss: 0.7036754488945007 Train_acc: 73.750000%
Test accuracy of the model: 48.000000%
Test accuracy of the model: 55.416667%
Train Epoch: 59/60 Train_Loss: 0.6709561347961426 Train_acc: 72.500000%
Train Epoch: 60/60 Train_Loss: 0.7036754488945007 Train_acc: 73.750000%
Test accuracy of the model: 48.000000%
Test accuracy of th

Train Epoch: 1/20 Train_Loss: 0.8952512145042419 Train_acc: 60.000000%
Train Epoch: 2/20 Train_Loss: 0.8398950695991516 Train_acc: 64.562500%
Train Epoch: 3/20 Train_Loss: 1.063136100769043 Train_acc: 67.937500%
Train Epoch: 4/20 Train_Loss: 0.9711516499519348 Train_acc: 67.937500%
Train Epoch: 3/20 Train_Loss: 1.063136100769043 Train_acc: 67.937500%
Train Epoch: 4/20 Train_Loss: 0.9711516499519348 Train_acc: 67.937500%
Train Epoch: 5/20 Train_Loss: 0.9054867029190063 Train_acc: 68.062500%
Train Epoch: 6/20 Train_Loss: 0.7593162059783936 Train_acc: 69.062500%
Train Epoch: 5/20 Train_Loss: 0.9054867029190063 Train_acc: 68.062500%
Train Epoch: 6/20 Train_Loss: 0.7593162059783936 Train_acc: 69.062500%
Train Epoch: 7/20 Train_Loss: 0.6856563091278076 Train_acc: 71.562500%
Train Epoch: 8/20 Train_Loss: 0.8102222681045532 Train_acc: 69.500000%
Train Epoch: 7/20 Train_Loss: 0.6856563091278076 Train_acc: 71.562500%
Train Epoch: 8/20 Train_Loss: 0.8102222681045532 Train_acc: 69.500000%
Train Ep

Train Epoch: 34/60 Train_Loss: 0.9548543095588684 Train_acc: 68.125000%
Train Epoch: 35/60 Train_Loss: 0.9114596247673035 Train_acc: 71.562500%
Train Epoch: 34/60 Train_Loss: 0.9548543095588684 Train_acc: 68.125000%
Train Epoch: 35/60 Train_Loss: 0.9114596247673035 Train_acc: 71.562500%
Train Epoch: 36/60 Train_Loss: 0.796680212020874 Train_acc: 71.250000%
Train Epoch: 37/60 Train_Loss: 0.8874531388282776 Train_acc: 72.187500%
Train Epoch: 36/60 Train_Loss: 0.796680212020874 Train_acc: 71.250000%
Train Epoch: 37/60 Train_Loss: 0.8874531388282776 Train_acc: 72.187500%
Train Epoch: 38/60 Train_Loss: 0.9075551629066467 Train_acc: 70.312500%
Train Epoch: 39/60 Train_Loss: 0.688866138458252 Train_acc: 70.625000%
Train Epoch: 38/60 Train_Loss: 0.9075551629066467 Train_acc: 70.312500%
Train Epoch: 39/60 Train_Loss: 0.688866138458252 Train_acc: 70.625000%
Train Epoch: 40/60 Train_Loss: 0.7241209149360657 Train_acc: 73.750000%
Train Epoch: 41/60 Train_Loss: 0.8529219031333923 Train_acc: 71.8750

Train Epoch: 25/40 Train_Loss: 0.4929722249507904 Train_acc: 77.708336%
Train Epoch: 26/40 Train_Loss: 0.6987049579620361 Train_acc: 77.708336%
Train Epoch: 27/40 Train_Loss: 0.480108380317688 Train_acc: 79.375008%
Train Epoch: 27/40 Train_Loss: 0.480108380317688 Train_acc: 79.375008%
Train Epoch: 28/40 Train_Loss: 0.5410069227218628 Train_acc: 80.833336%
Train Epoch: 28/40 Train_Loss: 0.5410069227218628 Train_acc: 80.833336%
Train Epoch: 29/40 Train_Loss: 0.7004603743553162 Train_acc: 81.250008%
Train Epoch: 30/40 Train_Loss: 0.38502997159957886 Train_acc: 81.250008%
Train Epoch: 29/40 Train_Loss: 0.7004603743553162 Train_acc: 81.250008%
Train Epoch: 30/40 Train_Loss: 0.38502997159957886 Train_acc: 81.250008%
Train Epoch: 31/40 Train_Loss: 0.6682068705558777 Train_acc: 82.500008%
Train Epoch: 32/40 Train_Loss: 0.4345167279243469 Train_acc: 81.875008%
Train Epoch: 31/40 Train_Loss: 0.6682068705558777 Train_acc: 82.500008%
Train Epoch: 32/40 Train_Loss: 0.4345167279243469 Train_acc: 81.

...

Train Epoch: 17/20 Train_Loss: 0.38588613271713257 Train_acc: 79.000000%
Train Epoch: 18/20 Train_Loss: 0.623914897441864 Train_acc: 79.187500%
Train Epoch: 17/20 Train_Loss: 0.38588613271713257 Train_acc: 79.000000%
Train Epoch: 18/20 Train_Loss: 0.623914897441864 Train_acc: 79.187500%
Train Epoch: 19/20 Train_Loss: 0.589214026927948 Train_acc: 79.250000%
Train Epoch: 20/20 Train_Loss: 0.5362969636917114 Train_acc: 78.937500%
Test accuracy of the model: 68.750000%
Train Epoch: 19/20 Train_Loss: 0.589214026927948 Train_acc: 79.250000%
Train Epoch: 20/20 Train_Loss: 0.5362969636917114 Train_acc: 78.937500%
Test accuracy of the model: 68.750000%
Test accuracy of the model: 74.166667%
Test accuracy of the model: 73.750000%
Training Time: 3.52 seconds
可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  4 TL:  0
---------------------------------------------------------------------------------------------------

Train Epoch: 48/60 Train_Loss: 0.5111786127090454 Train_acc: 78.750000%
Train Epoch: 49/60 Train_Loss: 0.6053162813186646 Train_acc: 76.250000%
Train Epoch: 50/60 Train_Loss: 0.6345410943031311 Train_acc: 76.875000%
Train Epoch: 51/60 Train_Loss: 0.5573155283927917 Train_acc: 77.500000%
Train Epoch: 50/60 Train_Loss: 0.6345410943031311 Train_acc: 76.875000%
Train Epoch: 51/60 Train_Loss: 0.5573155283927917 Train_acc: 77.500000%
Train Epoch: 52/60 Train_Loss: 0.5447379946708679 Train_acc: 76.875000%
Train Epoch: 53/60 Train_Loss: 0.5929170846939087 Train_acc: 77.812500%
Train Epoch: 52/60 Train_Loss: 0.5447379946708679 Train_acc: 76.875000%
Train Epoch: 53/60 Train_Loss: 0.5929170846939087 Train_acc: 77.812500%
Train Epoch: 54/60 Train_Loss: 0.4771769344806671 Train_acc: 76.875000%
Train Epoch: 55/60 Train_Loss: 0.45811495184898376 Train_acc: 78.437500%
Train Epoch: 54/60 Train_Loss: 0.4771769344806671 Train_acc: 76.875000%
Train Epoch: 55/60 Train_Loss: 0.45811495184898376 Train_acc: 7

Train Epoch: 40/40 Train_Loss: 0.39643624424934387 Train_acc: 87.291672%
Test accuracy of the model: 62.250000%
Test accuracy of the model: 78.750000%
Test accuracy of the model: 85.000000%
Training Time: 8.40 seconds
----------------------------------------------------------------------------------------------------------------
fold:  4 TL:  2
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/20 Train_Loss: 1.0519142150878906 Train_acc: 63.937500%
Train Epoch: 2/20 Train_Loss: 0.994698166847229 Train_acc: 68.375000%
Train Epoch: 1/20 Train_Loss: 1.0519142150878906 Train_acc: 63.937500%
Train Epoch: 2/20 Train_Loss: 0.994698166847229 Train_acc: 68.375000%
Train Epoch: 3/20 Train_Loss: 0.793165922164917 Train_acc: 70.687500%
Train Epoch: 4/20 Train_Loss: 0.6176310181617737 Train_acc: 73.125000%
Train Epoch: 3/20 Train_Loss: 0.793165922164917 Train_acc: 70.687500%
Train Epoch: 4/20 Train_Loss: 0.617631018161773

Train Epoch: 29/60 Train_Loss: 0.682918131351471 Train_acc: 67.812500%
Train Epoch: 30/60 Train_Loss: 0.8919488191604614 Train_acc: 70.937500%
Train Epoch: 31/60 Train_Loss: 0.8087841272354126 Train_acc: 70.937500%
Train Epoch: 32/60 Train_Loss: 0.8639541864395142 Train_acc: 69.062500%
Train Epoch: 31/60 Train_Loss: 0.8087841272354126 Train_acc: 70.937500%
Train Epoch: 32/60 Train_Loss: 0.8639541864395142 Train_acc: 69.062500%
Train Epoch: 33/60 Train_Loss: 0.7420541644096375 Train_acc: 69.062500%
Train Epoch: 34/60 Train_Loss: 0.8591046333312988 Train_acc: 70.937500%
Train Epoch: 33/60 Train_Loss: 0.7420541644096375 Train_acc: 69.062500%
Train Epoch: 34/60 Train_Loss: 0.8591046333312988 Train_acc: 70.937500%
Train Epoch: 35/60 Train_Loss: 0.7452183961868286 Train_acc: 73.750000%
Train Epoch: 36/60 Train_Loss: 0.631313145160675 Train_acc: 73.125000%
Train Epoch: 35/60 Train_Loss: 0.7452183961868286 Train_acc: 73.750000%
Train Epoch: 36/60 Train_Loss: 0.631313145160675 Train_acc: 73.125

Train Epoch: 21/40 Train_Loss: 0.6071505546569824 Train_acc: 79.791672%
Train Epoch: 22/40 Train_Loss: 0.5182166695594788 Train_acc: 80.729172%
Train Epoch: 23/40 Train_Loss: 0.5491015911102295 Train_acc: 82.500008%
Train Epoch: 24/40 Train_Loss: 0.5985963344573975 Train_acc: 82.708336%
Train Epoch: 23/40 Train_Loss: 0.5491015911102295 Train_acc: 82.500008%
Train Epoch: 24/40 Train_Loss: 0.5985963344573975 Train_acc: 82.708336%
Train Epoch: 25/40 Train_Loss: 0.27225157618522644 Train_acc: 83.541672%
Train Epoch: 26/40 Train_Loss: 0.49029579758644104 Train_acc: 81.250008%
Train Epoch: 25/40 Train_Loss: 0.27225157618522644 Train_acc: 83.541672%
Train Epoch: 26/40 Train_Loss: 0.49029579758644104 Train_acc: 81.250008%
Train Epoch: 27/40 Train_Loss: 0.45410534739494324 Train_acc: 83.125008%
Train Epoch: 27/40 Train_Loss: 0.45410534739494324 Train_acc: 83.125008%
Train Epoch: 28/40 Train_Loss: 0.6769522428512573 Train_acc: 81.250008%
Train Epoch: 29/40 Train_Loss: 0.5184752941131592 Train_ac

In [9]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()